In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [2]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
1328,basic_model_480_34,"(1, 480, 480, 3)",733.335791,480,691200,33133184,11008,570,79295508671,59,34,16,1,35042884,39594707232
1329,basic_model_256_13,"(1, 256, 256, 3)",705.425112,256,196608,2093972,3824,1010,109798665152,25,13,5,1,135107384,54848514304
1330,basic_model_350_6,"(1, 350, 350, 3)",698.543617,350,367500,172904,1912,810,42303275359,15,6,5,1,127277600,21112385000
1331,basic_model_64_21,"(1, 64, 64, 3)",621.550717,64,12288,25503568,7072,1210,59776080802,38,21,10,1,10843232,29858337280
1332,basic_model_400_14,"(1, 400, 400, 3)",602.228921,400,480000,8727042,3504,70200,74335462549,27,14,5,1,84037806,37114318592


In [32]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
88,EfficientNet-B0,"(512, 512, 3)",166.337081,512,786432,3965532,84032,1281000,4085726980,233,81,49,1,117635172,80094501376
89,EfficientNet-B0,"(720, 720, 3)",323.959066,720,1555200,3965532,84032,1281000,8183072596,233,81,49,1,233556068,161236938752
90,EfficientNet-B4,"(90, 90, 3)",67.848263,90,24300,17423416,250400,1793000,595640786,470,160,96,1,10284484,12812090224
91,EfficientNet-B4,"(512, 512, 3)",474.614049,512,786432,17423416,250400,1793000,15877844034,470,160,96,1,305708296,366198974816
92,EfficientNet-B4,"(720, 720, 3)",909.674533,720,1555200,17423416,250400,1793000,31863479154,470,160,96,1,607426440,737567732576


In [5]:
train_dataset.pop('input_width')
train_dataset.pop('nb_params_fc')
train_dataset.pop('nb_params_bn')
train_dataset.pop('nb_bn_layers')
train_dataset.pop('nb_fc_layers')
train_dataset.pop('nb_conv_layers')

0       28
1       28
2       28
3       28
4       28
        ..
1328    34
1329    13
1330     6
1331    21
1332    14
Name: nb_conv_layers, Length: 1333, dtype: int64

In [33]:
test_dataset.pop('input_width')
test_dataset.pop('nb_params_fc')
test_dataset.pop('nb_params_bn')
test_dataset.pop('nb_bn_layers')
test_dataset.pop('nb_fc_layers')
test_dataset.pop('nb_conv_layers')

0     160
1     160
2     200
3     200
4     200
     ... 
88     81
89     81
90    160
91    160
92    160
Name: nb_conv_layers, Length: 93, dtype: int64

In [7]:
Y = train_dataset.pop('ACET')
X = train_dataset
Y_f = Y
X_f = X

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [9]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [10]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [11]:
X_f_model_names = X_f.pop('Model_name')
X_f_input_shapes = X_f.pop('input_shape')
dtrain_f = xgb.DMatrix(X_f, label=Y_f)

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [34]:
Y_e = test_dataset.pop('ACET')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [14]:
params = {
    # Parameters that we are going to tune.
    'max_depth':12,
    'min_child_weight': 8,
    'eta':.05,
    'subsample': .8,
    'colsample_bytree': 1.,
    'gamma' : 0.5,
    # Other parameters
    'objective':'reg:squarederror',
}

In [15]:
params['eval_metric'] = "rmse"
num_boost_round = 1000

In [16]:
print(params)

{'max_depth': 12, 'min_child_weight': 8, 'eta': 0.05, 'subsample': 0.8, 'colsample_bytree': 1.0, 'gamma': 0.5, 'objective': 'reg:squarederror', 'eval_metric': 'rmse'}


In [17]:
num_boost_round = 70
best_model = xgb.train( 
    params,
    dtrain,
    num_boost_round=num_boost_round,
)

In [18]:
#best_model.save_model("model_16-04.model")
loaded_model = best_model

In [35]:
y_pred = loaded_model.predict(dtest_e)
np_test = Y_e.values
y_test = Y_e.values

In [36]:
# Model evaluation on test dataset
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error :', median_absolute_error(y_test, y_pred))

Explained variance :  0.9879340939899577
R2_value :  0.9879175624214821
Mean absolute error : 47.35168780422416
Root Mean squared error : 96.2894685274151
Median absolute error : 18.239778573437547


In [37]:
# Calculate mean absolute percentage error (MAPE)
np_test = y_test
errors = abs(y_pred-np_test)
mape = 100 * (errors / np_test) # Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('MAX MAPE : ',max(mape))
print('Mape:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE :  45.05327504738231
Mape: 9.12 %.
Accuracy: 90.88 %.


In [38]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [39]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, np_test[i], y_pred[i],abs(y_pred[i]-np_test[i]), ((abs(y_pred[i]-np_test[i]))/np_test[i])*100])